#**스마트폰 센서 데이터 기반 모션 분류**
# 단계3 : 단계별 모델링


## 0.미션

단계별로 나눠서 모델링을 수행하고자 합니다.  

* 단계1 : 정적(0), 동적(1) 행동 분류 모델 생성
* 단계2 : 세부 동작에 대한 분류모델 생성
    * 단계1 모델에서 0으로 예측 -> 정적 행동 3가지 분류 모델링
    * 단계1 모델에서 1으로 예측 -> 동적 행동 3가지 분류 모델링 
* 모델 통합
    * 두 단계 모델을 통합하고, 새로운 데이터에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
* 성능 비교
    * 기본 모델링의 성능과 비교
    * 모든 모델링은 [다양한 알고리즘 + 성능 튜닝]을 수행해야 합니다.


## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.





In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
train = pd.read_csv('/content/drive/MyDrive/2023/AIVLE/miniproject5/train_dataset.csv')
test = pd.read_csv('/content/drive/MyDrive/2023/AIVLE/miniproject5/test_dataset.csv')
submission = pd.read_csv('/content/drive/MyDrive/2023/AIVLE/miniproject5/sample_submission.csv')

### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용

 <br/>  

* 세부 요구사항
    - data01_train.csv 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - data01_test.csv 를 불러와 'new_data' 이름으로 저장합니다.


In [18]:
data = pd.read_csv('/content/drive/MyDrive/2023/AIVLE/miniproject5/2023.04.12_미니프로젝트5차_3_5일차 실습자료/data01_train.csv')
features = pd.read_csv('/content/drive/MyDrive/2023/AIVLE/miniproject5/2023.04.12_미니프로젝트5차_3_5일차 실습자료/features.csv')

In [19]:
data = data.drop(['subject'], axis = 1)

In [20]:
data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.487737,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,STANDING
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.237820,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,LAYING
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,-0.535287,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,STANDING
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,-0.004012,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388,WALKING
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.157832,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758,WALKING_DOWNSTAIRS


## 2.데이터 전처리

In [21]:
label_dict = {
    'STANDING': 0,
    'LAYING': 1,
    'WALKING': 2,
    'WALKING_DOWNSTAIRS': 3,
    'WALKING_UPSTAIRS': 4,
    'SITTING': 5
}

y = data.replace({'Activity' : label_dict}).copy()['Activity']

data['target'] = y

In [22]:
X = data.drop(['Activity', 'target'], axis = 1) #'subject'
y = data['target']

### (2) 스케일링(필요시)


* 세부 요구사항
    - 스케일링을 필요로 하는 알고리즘 사용을 위해서 코드 수행
    - min-max 방식 혹은 standard 방식 중 한가지 사용.

In [23]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 선언 및 Fitting
mMscaler = MinMaxScaler()
mMscaler.fit(X)

# 데이터 변환
mMscaled_data = mMscaler.transform(X)

# 데이터 프레임으로 저장
X_sc = pd.DataFrame(mMscaled_data, columns = X.columns)

ValueError: ignored

In [90]:
# # 모듈 불러오기
# from sklearn.model_selection import train_test_split

# # 7:3으로 분리
# X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=0.3, random_state=1)  

* 세부 요구사항
    - Label 추가 : data 에 Activity_dynamic 를 추가합니다. Activity_dynamic은 과제1에서 is_dynamic과 동일한 값입니다.
    - x와 y1, y2로 분할하시오.
        * y1 : Activity
        * y2 : Activity_dynamic
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [11]:
df = X_sc.copy()
df['target'] = data['target']

#df['Activity_dynamic'] = df['target'].apply(lambda x : 0 if x == 'STANDING' or x == 'SITTING' or x == 'LAYING' else 1)
df1 = df[(df['target'] == 0) | (df['target'] == 1) | (df['target'] == 2)]#.reset_index(drop=True)
df2 = df[(df['target'] == 3) | (df['target'] == 4) | (df['target'] == 5)]#.reset_index(drop=True)

In [12]:
df2

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",target
4,0.597564,0.426728,0.545615,0.544245,0.466164,0.305633,0.494765,0.452314,0.326814,0.792066,...,0.417223,0.223002,0.471641,0.077366,0.012683,0.056187,0.149176,0.855762,0.564555,3
5,0.554940,0.410978,0.469314,0.392120,0.574568,0.536397,0.365072,0.538604,0.550649,0.490368,...,0.192683,0.057620,0.478571,0.073954,0.250065,0.654328,0.226198,0.848275,0.642074,4
11,0.523064,0.397787,0.458296,0.002245,0.008030,0.048618,0.002161,0.007902,0.055953,0.031927,...,0.324036,0.131299,0.482037,0.466816,0.538498,0.575653,0.057100,0.709899,0.537925,5
14,0.542315,0.414629,0.430016,0.617788,0.711912,0.449592,0.609605,0.676844,0.446266,0.637800,...,0.342301,0.222602,0.461916,0.313908,0.982338,0.910749,0.146944,0.878658,0.507251,4
16,0.606190,0.374606,0.419665,0.339365,0.474941,0.586495,0.302860,0.459601,0.602875,0.506912,...,0.375259,0.181520,0.193757,0.773891,0.768140,0.460167,0.099765,0.835120,0.501923,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5872,0.528375,0.392226,0.441631,0.011313,0.018790,0.018584,0.011185,0.020354,0.018873,0.041587,...,0.239300,0.070068,0.542742,0.434313,0.539170,0.482282,0.349360,0.526929,0.330120,5
5876,0.519354,0.399138,0.434013,0.002977,0.030931,0.041172,0.002869,0.033852,0.045956,0.031890,...,0.342081,0.158131,0.476425,0.369282,0.304459,0.061314,0.044320,0.753647,0.535461,5
5877,0.462416,0.414401,0.447413,0.436516,0.458287,0.336149,0.403739,0.452190,0.352357,0.539322,...,0.181148,0.061611,0.859210,0.811576,0.934035,0.277715,0.156635,0.881634,0.472671,4
5879,0.476525,0.377925,0.439500,0.479313,0.549190,0.207188,0.449643,0.519943,0.222646,0.609907,...,0.597411,0.483596,0.626061,0.775871,0.028132,0.068685,0.142611,0.874641,0.507571,4


## **3.단계별 모델링**

![](https://github.com/DA4BAM/image/blob/main/step%20by%20step.png?raw=true)

### (1) 단계1 : 정적/동적 행동 분류 모델

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)과 동적 행동(동적 : Walking, Walking-Up, Walking-Down)을 구분하는 모델 생성.
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [13]:
df1_X = df1.drop(['target'], axis = 1)
df1_y = df1['target']

#### 1) 알고리즘1 : 

##### 정적

In [14]:
# 1단계: 불러오기
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
from sklearn.model_selection import cross_val_score


In [15]:
# 학습/예측/평가 수행. 
dt = RandomForestClassifier()
# 성능 지표는 정확도(accuracy) , 교차 검증 세트는 5개 
scores = cross_val_score(dt , df1_X, df1_y, scoring='accuracy',cv=5)

print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores),4))
# # 5단계 평가하기
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))

교차 검증별 정확도: [0.9984 1.     1.     1.     1.    ]
평균 검증 정확도: 0.9997


In [16]:
dt.fit(df1_X, df1_y)
pred1 = dt.predict(df1_X)

In [98]:
df1['pred1'] = pred1

<ipython-input-98-f797c221255d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['pred1'] = pred1


In [99]:
df1

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",target,pred1
0,0.526878,0.401033,0.448319,0.005507,0.019335,0.016125,0.005500,0.018720,0.017175,0.035126,...,0.093564,0.472577,0.477891,0.654361,0.537449,0.201725,0.900647,0.578396,0,0
1,0.511749,0.396653,0.450918,0.005224,0.002718,0.006117,0.004906,0.002821,0.006221,0.031332,...,0.156521,0.462253,0.694229,0.117569,0.887502,0.680309,0.156150,0.420272,1,1
2,0.520362,0.397878,0.445641,0.001140,0.009789,0.002830,0.001033,0.008760,0.002492,0.028708,...,0.087117,0.494210,0.237489,0.054098,0.511768,0.084172,0.813468,0.478444,0,0
3,0.527733,0.385400,0.424823,0.384137,0.518471,0.330830,0.363222,0.515313,0.326042,0.504144,...,0.301955,0.365002,0.806402,0.874258,0.464438,0.153833,0.870783,0.551238,2,2
6,0.415799,0.387557,0.497402,0.423901,0.462788,0.380185,0.398800,0.424345,0.376450,0.557334,...,0.114476,0.719316,0.498256,0.029927,0.521717,0.057348,0.793784,0.492144,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5871,0.522751,0.397204,0.444912,0.029368,0.009967,0.004938,0.029606,0.010125,0.005777,0.056926,...,0.081560,0.536932,0.604997,0.710482,0.413125,0.968528,0.381689,0.241164,1,1
5873,0.511219,0.385884,0.455930,0.315921,0.482928,0.385095,0.302807,0.466163,0.378238,0.369456,...,0.487377,0.513057,0.048154,0.807991,0.194923,0.201480,0.903825,0.574030,2,2
5874,0.506243,0.160858,0.508687,0.040167,0.561095,0.184506,0.031007,0.597949,0.193245,0.066852,...,0.464150,0.493675,0.633595,0.407163,0.550128,0.906757,0.653503,0.026258,1,1
5875,0.519476,0.398598,0.447839,0.001702,0.006447,0.013163,0.001814,0.005934,0.013244,0.028922,...,0.022245,0.555961,0.568137,0.145906,0.246589,0.091910,0.827125,0.512879,0,0


##### 동적

In [65]:
df2_X = df2.drop(['target'], axis = 1)
df2_y = df2['target']

In [66]:
# 학습/예측/평가 수행. 
dt = DecisionTreeClassifier()
# 성능 지표는 정확도(accuracy) , 교차 검증 세트는 5개 
scores = cross_val_score(dt , df2_X, df2_y, scoring='accuracy',cv=5)

print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores),4))
# # 5단계 평가하기
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))

교차 검증별 정확도: [0.9683 0.9795 0.9683 0.9776 0.9664]
평균 검증 정확도: 0.972


#### 2) 알고리즘2 : 

In [100]:
label_dict = {
    3: 0,
    4: 1,
    5: 2
}

modeling_target = df2.replace({'target' : label_dict}).copy()['target']

In [101]:
import xgboost as xgb 
from sklearn.model_selection import cross_val_score

xg = xgb.XGBClassifier()

# 성능 지표는 정확도(accuracy) , 교차 검증 세트는 5개 
scores = cross_val_score(xg , df2_X, modeling_target, scoring='accuracy',cv=5)

print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores),4))

교차 검증별 정확도: [0.9963 0.9944 0.9925 1.     0.9963]
평균 검증 정확도: 0.9959


In [102]:
xg.fit(df2_X, modeling_target)
pred2 = xg.predict(df2_X)

In [103]:
df2['pred2'] = pred2

<ipython-input-103-3c5e092b0557>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['pred2'] = pred2


In [104]:
label_dict = {
    0: 3,
    1: 4,
    2: 5
}

pred2 = df2.replace({'pred2' : label_dict}).copy()['pred2']


In [105]:
df2['pred2'] = pred2

<ipython-input-105-3c5e092b0557>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['pred2'] = pred2


In [107]:
df2['pred2']

4       3
5       4
11      5
14      4
16      4
       ..
5872    5
5876    5
5877    4
5879    4
5880    5
Name: pred2, Length: 2681, dtype: int64

## 데이터프레임 합치기

In [108]:
df1['pred1']

0       0
1       1
2       0
3       2
6       2
       ..
5871    1
5873    2
5874    1
5875    0
5878    1
Name: pred1, Length: 3200, dtype: int64

In [115]:
df2['pred1']

4       3
5       4
11      5
14      4
16      4
       ..
5872    5
5876    5
5877    4
5879    4
5880    5
Name: pred1, Length: 2681, dtype: int64

In [121]:
temp = pd.concat([df1['pred1'],df2['pred2']],axis=1, join='outer').sort_index()

In [125]:
temp = temp.fillna(0)

In [128]:
pred = temp['pred1']+temp['pred2']
pred.astype(int)

0       0
1       1
2       0
3       2
4       3
       ..
5876    5
5877    4
5878    1
5879    4
5880    5
Length: 5881, dtype: int64

### (2) 단계2-1 : 정적 동작 세부 분류

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)인 데이터 추출
    * Laying, Sitting, Standing 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

### (3) 단계2-2 : 동적 동작 세부 분류

* 세부 요구사항
    * 동동적 행동(Walking, Walking Upstairs, Walking Downstairs)인 데이터 추출
    * Walking, Walking Upstairs, Walking Downstairs 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

### (4) 분류 모델 합치기


* 세부 요구사항
    * 두 단계 모델을 통합하고, 새로운 데이터(test)에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
    * 데이터 파이프라인 구축 : test데이터가 로딩되어 전처리 과정을 거치고, 예측 및 성능 평가 수행

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

#### 1) 함수 만들기